## Example of training/testing using BNN with bayesian-torch

In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path
from types import SimpleNamespace
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from bnnrul.cmapss.dataset import CMAPSSDataModule
from bnnrul.cmapss.models import CMAPSSModel, CMAPSSModelBnn, get_checkpoint, TBLogger

In [3]:
args = SimpleNamespace(
    data_path="../data/cmapss",
    out_path="../results/cmapss",
    scn="bnn_bt",
    net="linear",
)
checkpoint_dir = Path(f"{args.out_path}/{args.scn}/checkpoints/{args.net}")
logger = TBLogger(
    f"{args.out_path}/{args.scn}/lightning_logs/{args.net}",
    default_hp_metric=False,
)

### Train

In [4]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
train_dl = data.train_dataloader()

In [5]:
const_bnn_prior_parameters = {
    "prior_mu": 0.0,
    "prior_sigma": 1.0,
    "posterior_mu_init": 0.0,
    "posterior_rho_init": -3.0,
    "type": "Reparameterization",  # Flipout or Reparameterization
    "moped_enable": False,  # True to initialize mu/sigma from the pretrained dnn weights
    "moped_delta": 0.5,
}
bnn = CMAPSSModelBnn(
    data.win_length,
    data.n_features,
    args.net,
    const_bnn_prior_parameters,
    num_mc_samples_elbo=10,
    num_predictions=10,
)
bnn

CMAPSSModelBnn(
  (net): Linear(
    (layers): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): LinearReparameterization()
      (2): Sigmoid()
      (3): LinearReparameterization()
      (4): Sigmoid()
      (5): LinearReparameterization()
      (6): Sigmoid()
      (7): LinearReparameterization()
      (8): Softplus(beta=1, threshold=20)
    )
  )
)

In [6]:
checkpoint_file = get_checkpoint(checkpoint_dir)
monitor = "loss/val"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir, monitor=monitor)
earlystopping_callback = EarlyStopping(monitor=monitor, patience=20)

trainer = pl.Trainer(
    gpus=[0],
    max_epochs=12,
    log_every_n_steps=10,
    resume_from_checkpoint=checkpoint_file,
    logger=logger,
    callbacks=[
        checkpoint_callback,
        earlystopping_callback,
    ],
)
trainer.fit(bnn, data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Missing logger folder: ../results/cmapss/bnn_bt/lightning_logs/linear/default

  | Name | Type   | Params
--------------------------------
0 | net  | Linear | 91.2 K
--------------------------------
91.2 K    Trainable params
0         Non-trainable params
91.2 K    Total params
0.365     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

### Test

In [7]:
data = CMAPSSDataModule(args.data_path, batch_size=1)
test_dl = data.test_dataloader()

In [8]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
bnn = CMAPSSModelBnn.load_from_checkpoint(
    get_checkpoint(checkpoint_dir), num_predictions=10
)
trainer = pl.Trainer(
    gpus=[0],
    log_every_n_steps=10,
    logger=logger,
)
trainer.test(bnn, data, verbose=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: 0it [00:00, ?it/s]

[{'loss/test': 6788.51416015625,
  'loss_mse/test': 6778.53564453125,
  'loss_kl/test': 9.9784574508667}]